## Kinome Atlas

In [2]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import re
import urllib
import numpy as np

In [3]:
kinome = []
total_kinome = []
for page in range(1,17):
    print("page "+str(page),end="\r")
    url = 'https://kinase-atlas.bu.edu/uniprot/index?page='+str(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    for row in soup.table.select("tbody tr"):
        tds = [td.get_text(strip=True) for td in row.select("td")]
    #     print(tds)
        if tds[2] == "Homo sapiens":
            kinome.append([tds[0],tds[1]])
        total_kinome.append(tds)

In [4]:
len(kinome)

240

## Uniprot

In [5]:
def word_break(text_br):
    words = re.findall(r'\b\S+\b', text_br) 
    return words

# homo sapiens, reviewed, family=kinases
filename = "uniprot-family_kinase.txt"
doc = open(filename, "r")
text_string = doc.read()
match_word=word_break(text_string)


In [6]:
kinases = []
kinases_pdb = []
ac_uniprot = ""
id_uniprot = ""
method = ""
resolution = 10000
pdb = ""
first_time = False
for i in range(len(match_word)):
    
        if match_word[i] == "ID":
            kinases.append([ ac_uniprot,id_uniprot,method,pdb,resolution])
            if pdb != "" :
                kinases_pdb.append([ ac_uniprot,id_uniprot,method,pdb,resolution])
            first_time = True
            # from previous protein
            method = ""
            pdb = ""
            resolution = 10000
            
            id_uniprot =  match_word[i+1] 
        
        
        if match_word[i] == "AC":
            if first_time:
                ac_uniprot =  match_word[i+1]
                first_time = False
#                 kinases.append([ ac_uniprot,id_uniprot])
            
        if match_word[i] == "PDB" and match_word[i+1] != "data":
            if method == "":
                method = match_word[i+2]
                pdb = match_word[i+1]
                if method != "NMR" and method != "Model" and method != "Other":
#                     print(match_word[i+1],match_word[i+2],match_word[i+3])
                    resolution = float(match_word[i+3])
                    
            elif match_word[i+2] != "NMR" and match_word[i+2] != "Model" and match_word[i+2] != "Other":
#                 print(match_word[i+1],match_word[i+2],match_word[i+3])
                if resolution < float(match_word[i+3]):
                    method = match_word[i+2]
                    pdb = match_word[i+1]  
                    resolution = float(match_word[i+3])


       

In [7]:
len(kinases_pdb)

402

## Comparison Kinome Atlas, Uniprot

In [8]:
kinases_pdb = np.array(kinases_pdb)
uniprot_kinases = kinases_pdb[:,0]

kinome = np.array(kinome)
atlas_kinases = kinome[:,0]

In [9]:
len(atlas_kinases)

240

In [10]:
len(uniprot_kinases)

402

In [11]:
counter = 0
for kinase in atlas_kinases:
    if kinase in uniprot_kinases:
        counter += 1
    else:
        print(kinase)
print(str(counter)+"/"+str(len(atlas_kinases)))

D3DSX2
239/240


In [ ]:
# https://www.uniprot.org/uniprot/D3DSX2
    
# https://www.uniprot.org/uniprot/P11362



### Check for duplicates

In [14]:
kinases_pdb_uniq = np.array(kinases_pdb)
uniprot_kinases_uniq = kinases_pdb_uniq[:,3]

In [16]:
def checkIfDuplicates(listOfElems):
    ''' Check if given list contains any duplicates '''    
    for elem in listOfElems:
        if listOfElems.count(elem) > 1:
            print()
            return True
    return False

In [20]:
checkIfDuplicates(list(uniprot_kinases_uniq))

True

There are duplicates

In [23]:
mylist = list(uniprot_kinases_uniq)
mylist = list(dict.fromkeys(mylist))
print(len(mylist))

396
